In [64]:
import helper
import mne
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
from pathlib import Path, PurePath

%matplotlib
plt.rcParams.update({'font.size': 17})

Using matplotlib backend: Qt5Agg


In [107]:
# Parameters 

# Subject and task
subject = 'DiAs'
subject_id = '04'
proc = 'BP'
task = 'rest_baseline'
run = '1'

# High frequency bands 
l_freq = 60
nband = 6 
band_size = 20 

# electrodes to pick 

picks = ['LGRD60-LGRD61', 'LGRD49-LGRD57', 'LGRD50-LGRD51', 'LGRD58-LGRD59','LTo1-LTo2','LTo3-LTo4','LTo5-LTo6','LTp1-LTp2','LTp3-LTp4']

# Epoch parameter

tmin = -0.5 # Prestimulus
tmax = 1.75 # Poststimulus

# Saving paramerters 

save2 = Path('~','projects','CIFAR','data_fun').expanduser()
task_save = 'stimuli'
suffix_place = '_epoch_place.mat'
suffix_face = '_epoch_face.mat'

In [108]:
raw, dfelec = helper.import_data(task=task)

Reading /home/guime/CIFAR_data/iEEG_10/subjects/DiAs/EEGLAB_datasets/bipolar_montage/DiAs_freerecall_rest_baseline_1_preprocessed_BP_montage.fdt
Reading 0 ... 104998  =      0.000 ...   209.996 secs...


/home/guime/projects/CIFAR/new_code/test/helper.py:26: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(fpath, preload=load)
/home/guime/projects/CIFAR/new_code/test/helper.py:26: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(fpath, preload=load)


In [109]:
    HFB, raw_HFB = helper.HFB(raw, l_freq=60, nband=6, band_size=20);

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 80 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)

Converting "time" to "<class 'numpy.int64'>"...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1e+02 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)



/home/guime/projects/CIFAR/new_code/test/helper.py:86: RuntimeWarning: invalid value encountered in true_divide
  HFB = np.divide(HFB, np.mean(HFB, axis=0))


Converting "time" to "<class 'numpy.int64'>"...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1.2e+02 - 1.4e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)

Converting "time" to "<class 'numpy.int64'>"...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1.4e+02 - 1.6e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)

Conver

In [92]:
# Create annotations from initial raw
raw_HFB.set_annotations(raw.annotations)
events, events_id = mne.events_from_annotations(raw_HFB)
place_id, face_id = helper.stim_id(events_id)
epochs = mne.Epochs(raw_HFB, events, tmin=tmin, tmax=tmax, picks=picks, preload=True)
time = epochs.times

Used Annotations descriptions: ['-1', '-2', 'boundary']
3 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 3 events and 1126 original time points ...
1 bad epochs dropped


In [6]:
 HFB_db = helper.HFB_norm(epochs, picks, events, tmin)

58 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


In [ ]:
picks = ['LGRD60-LGRD61', 'LGRD50-LGRD51', 'LGRD58-LGRD59','LTo1-LTo2','LTo3-LTo4','LTo5-LTo6','LTp1-LTp2','LTp3-LTp4']

In [65]:
for index, channel in enumerate(['LGRD60-LGRD61', 'LTo3-LTo4']):
    plt.subplot(2,1,index+1)
    for condition in (face_id, place_id):
        HFB_condition = HFB_db[condition]
        evok = HFB_condition.average()
        evok = evok.pick(channel).data
        evok_std = HFB_condition.standard_error()
        evok_std = evok_std.pick(channel).data
        if condition[0]=='10':
            cdt = 'place'
        else: 
            cdt = 'face'
        plt.plot(time, evok[0,:], label = cdt)
        plt.fill_between(time, evok[0,:]-evok_std[0,:], evok[0,:]+evok_std[0,:])
        plt.ylabel('Amplitude (db)')
        plt.title(channel)
        plt.legend()
        plt.axvline(x=0)
plt.xlabel('Time (s)')

Text(0.5, 0, 'Time (s)')

In [76]:
# Plot psd 
HFB = raw_HFB.copy().get_data(picks='LGRD60-LGRD61')

In [110]:
from mne.time_frequency import psd_multitaper
chan = 'LGRD60-LGRD61'
psds, freqs = psd_multitaper(raw_HFB, low_bias=True,
                             fmin=0.01, fmax=10, proj=True, picks=chan,
                             n_jobs=1)
psds = 10 * np.log10(psds)
plt.plot(freqs, psds[0,:],label='rest')
plt.xscale('log')
plt.xlabel('Frequency (Hz), log scale')
plt.ylabel('Amplitude (dB), log scale')
plt.title('HFB spectra, categroy selective electrode')
plt.legend()

No projector specified for this dataset. Please consider the method self.add_proj.
    Using multitaper spectrum estimation with 7 DPSS windows
